<a href="https://colab.research.google.com/github/roberthsu2003/machine_learning/blob/main/%E7%B0%A1%E5%96%AE%E7%B7%9A%E6%80%A7%E8%BF%B4%E6%AD%B8/%E7%B7%9A%E6%80%A7%E8%BF%B4%E6%AD%B8%E5%85%AC%E5%BC%8F%E7%B9%AA%E5%9C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 步驟1: 安裝必要的套件
# 使用 %%capture 隱藏安裝過程的輸出訊息，讓介面更乾淨
# wget 是用來下載檔案的套件，我們需要它來下載中文字體檔案
%%capture
%pip install wget

UsageError: Line magic function `%%capture` not found.


In [3]:

# 步驟2: 下載中文字體檔案
# 這個步驟確保我們有中文字體來正確顯示中文標籤和標題

import wget  # 用於下載檔案的套件
import os    # 用於檔案系統操作的套件

# 檢查字體檔案是否已存在 - 避免重複下載
font_path = "ChineseFont.ttf"  # 定義字體檔案路徑

if not os.path.exists(font_path):  # 如果檔案不存在
    print("字體檔案不存在，正在下載...")
    try:
        # 從 GitHub 下載中文字體檔案
        wget.download("https://github.com/roberthsu2003/machine_learning/raw/refs/heads/main/source_data/ChineseFont.ttf")
        print(f"\n字體檔案下載完成: {font_path}")
    except Exception as e:  # 處理下載錯誤
        print(f"下載字體檔案時發生錯誤: {e}")
        print("將使用系統預設字體")
else:
    print(f"字體檔案已存在: {font_path}")  # 檔案已存在，無需下載

字體檔案已存在: ChineseFont.ttf


# 📊 簡單線性迴歸視覺化教學

## 🎯 **學習目標**
使用 NumPy 和 Matplotlib 來繪製簡單線性迴歸的程式碼。這個範例會包含：

### **主要步驟**
1. **📦 套件導入與環境設定** - 導入必要的套件並設定中文字體
2. **📈 數據生成** - 產生模擬的線性迴歸數據
3. **🎨 視覺化函數** - 建立互動式繪圖函數
4. **🎛️ 互動控制** - 創建滑桿來調整參數
5. **📊 圖表展示** - 顯示最終的互動式圖表

### **核心概念**
- **線性迴歸公式**: `y = wX + b`
  - `w`: 斜率 (slope/weight) - 控制線的傾斜程度
  - `b`: 截距 (bias/intercept) - 控制線與Y軸的交點
  - `X`: 特徵變數 (feature)
  - `y`: 目標變數 (target/label)

### **教學重點**
- 理解線性迴歸的基本概念
- 學習如何視覺化數據和模型
- 體驗參數調整對模型的影響

In [4]:
# 步驟3: 套件導入與環境設定
# 導入所有必要的套件來進行數據處理和視覺化

import numpy as np                    # 數值計算套件，用於生成和處理數據
import matplotlib.pyplot as plt       # 繪圖套件，用於創建圖表
import matplotlib as mpl             # matplotlib 的主要模組
from matplotlib.font_manager import fontManager  # 字體管理器
from ipywidgets import interactive    # 互動式控制項套件
import ipywidgets as widgets          # 小工具套件，用於創建滑桿等控制項

# 步驟4: 設定中文字體 - 確保中文標籤能正確顯示
try:
    if os.path.exists("ChineseFont.ttf"):
        fontManager.addfont("ChineseFont.ttf")  # 添加中文字體到字體管理器
        mpl.rc('font', family="ChineseFont")   # 設定預設字體為中文字體
        print("中文字體設定成功")
    else:
        print("中文字體檔案不存在，使用系統預設字體")
        # 可以設定其他中文字體作為備選
        mpl.rc('font', family=['DejaVu Sans', 'SimHei', 'Microsoft YaHei'])
except Exception as e:
    print(f"字體設定失敗: {e}")
    print("使用系統預設字體")
    mpl.rc('font', family=['DejaVu Sans', 'SimHei', 'Microsoft YaHei'])

# 步驟5: 設定隨機種子 - 確保每次執行的結果都相同，便於教學
np.random.seed(42)  # 42 是一個常用的隨機種子值

# 步驟6: 生成模擬數據
# 使用 linspace 在 0 到 10 之間產生 100 個均勻分布的 X 值
X = np.linspace(0, 10, 100)  # 創建 100 個從 0 到 10 的均勻分布點

# 定義真實的線性迴歸參數
w_true = 2.0  # 真實斜率 - 這是要學習的參數
b_true = 1.0  # 真實截距 - 這是要學習的參數

# 生成帶有雜訊的 y 值：y = w_true * X + b_true + 雜訊
# np.random.normal(0, 1, 100) 產生平均值為 0，標準差為 1 的 100 個隨機數
y = w_true * X + b_true + np.random.normal(0, 1, 100)

# 步驟7: 定義互動式繪圖函數
def plot_regression(y, w, b):
    """
    繪製線性迴歸圖表的函數
    
    參數:
    - y: 目標變數數據
    - w: 斜率參數
    - b: 截距參數
    """
    
    # 建立新的圖表，設定大小為 10x6 英寸
    plt.figure(figsize=(10, 6))

    # 繪製散佈圖 - 顯示原始數據點
    # 註解掉是因為在互動式教學中，我們主要關注回歸線的變化
    # plt.scatter(X, y, color='blue', alpha=0.5, label='數據點')

    # 繪製回歸線 - 根據當前參數 w 和 b 繪製直線
    plt.plot(X, w * X + b, color='red', linewidth=2, label='回歸線')

    # 設定圖表標題和軸標籤
    plt.title(f'簡單線性迴歸示例 (斜率={w:.1f}, 截距={b:.1f})', fontsize=14)
    plt.xlabel('X (特徵變數)', fontsize=12)
    plt.ylabel('y (目標變數)', fontsize=12)
    
    # 設定座標軸範圍
    plt.xlim(0, 10)   # X 軸範圍：0 到 10
    plt.ylim(-5, 15)  # Y 軸範圍：-5 到 15
    
    # 添加圖例和網格
    plt.legend(fontsize=12)  # 顯示圖例
    plt.grid(True, alpha=0.3)  # 顯示網格，透明度為 0.3
    plt.show()

# 步驟8: 創建互動式控制項
# 使用 interactive 函數創建可以調整參數的互動式圖表
interactive_plot = interactive(
    plot_regression,  # 要互動的函數
    y=y,              # 固定參數：數據
    w=widgets.FloatSlider(  # 斜率滑桿
        value=0,           # 初始值
        min=-5,           # 最小值
        max=5,            # 最大值
        step=0.1,         # 步長
        description='斜率 (w):',  # 滑桿標籤
        continuous_update=False   # 不連續更新，提升性能
    ),
    b=widgets.FloatSlider(  # 截距滑桿
        value=1,           # 初始值
        min=-5,           # 最小值
        max=5,            # 最大值
        step=0.1,         # 步長
        description='截距 (b):',  # 滑桿標籤
        continuous_update=False   # 不連續更新，提升性能
    )
)

# 步驟9: 顯示互動式圖表
# 執行這行會顯示帶有滑桿控制的圖表
interactive_plot

中文字體設定成功


interactive(children=(Dropdown(description='y', options=(np.float64(1.4967141530112327), np.float64(1.06375590…



## 📚 **程式碼執行結果說明**

### **🎯 程式碼功能**
這段程式碼會執行以下操作：

1. **📊 數據生成**
   - 使用 `np.linspace()` 在 0 到 10 之間產生 100 個均勻分布的點
   - 設定真實的斜率 `w = 2.0` 和截距 `b = 1.0`
   - 加入隨機雜訊來模擬真實世界的數據不確定性

2. **🎨 視覺化元素**
   - 使用紅色線顯示回歸線 (在互動模式中主要關注點)
   - 藍色散佈點代表原始數據 (可選擇性顯示)
   - 加入圖例、標題和座標軸標籤

3. **🎛️ 互動功能**
   - 提供滑桿來調整斜率 (w) 和截距 (b) 參數
   - 即時更新圖表，觀察參數變化對回歸線的影響

### **📈 預期結果**
執行這段程式碼後，您將看到：

- **🎯 互動式圖表**：包含滑桿控制的線性迴歸視覺化
- **📊 回歸線**：紅色直線代表當前參數下的回歸模型
- **🎛️ 控制面板**：兩個滑桿分別控制斜率和截距
- **📝 動態標題**：顯示當前的參數值

### **🔍 學習重點**
- **參數理解**：觀察斜率如何影響線的傾斜程度
- **截距影響**：了解截距如何改變線與Y軸的交點
- **模型擬合**：嘗試調整參數使回歸線更好地擬合數據
- **線性關係**：理解 `y = wX + b` 公式的視覺化表現

### **💡 教學建議**
1. **先觀察**：讓學生觀察預設參數下的回歸線
2. **調整斜率**：移動斜率滑桿，觀察線的傾斜變化
3. **調整截距**：移動截距滑桿，觀察線的上下移動
4. **最佳擬合**：嘗試找到最接近真實參數 (w=2, b=1) 的設定